# Graphing module

## Mapping and syntax processing (AST)

### Syntax processing
- Here there are 3 main options available (that I currently know of)
    - ANTLR (more community support)
    - tree-sitter (speed, lazy-evaluation)
    - Bison (more for config files)

Because currently we have the most done on tree-sitter, the focus will be on that, but it's always possible to switch or write a custom AST Parser since our usecase is quite unique

In [18]:
from dataclasses import dataclass
import sys

# This is used to be able to import from src directory
sys.path.append("../")


from src.repository_processing import files_from_repository
from tree_sitter_languages import get_language, get_parser
from tree_sitter import Language, Parser, Node
from programming_language_extensions import currated_language_list

#TODO: Add Result monad to properties to handle errors
# @property is not cached, if more performance needed use @functools.cached_property
@dataclass
class LanguageInfo:
    """holds language information and parser for AST"""
    name: str

    @property
    def extensions(self) -> tuple[str]:
        return currated_language_list[self.name]

    @property
    def language(self) -> Language:
        return get_language(self.name)
    
    @property
    def parser(self) -> Parser:
        return get_parser(self.name)

python_language = LanguageInfo("python")

import networkx as nx
from pyvis.network import Network

def print_tree_structure(node: Node, graph: nx.Graph):
    # Print the node's value
    print(node.id, node.type, node.text)
    
    # Add the node to the graph
    graph.add_node(node.id, title=str(node.text))
    
    # Recursively print the structure of the node's children
    for child in node.children:
        print_tree_structure(child, graph)
        graph.add_edge(node.id, child.id)
        


files = files_from_repository("https://github.com/Foxicution/repo-review")
for file in files:
    if file.path.endswith(python_language.extensions):
        print(file.path)
        # Parse the file and get the root node of the tree
        tree = python_language.parser.parse(file.content)
        root_node = tree.root_node

        # Create an empty graph
        graph = nx.Graph()

        # Walk through the tree and print its structure
        print_tree_structure(root_node, graph)

        # Draw the graph
        nt = Network('1000px', '2000px')
        nt.options.physics.enabled = False
        nt.show_buttons()
        # populates the nodes and edges data structures
        nt.from_nx(graph)
        nt.show('nx.html')
        break



/main.py
94155664235408 module b'import json\nimport os\nimport pickle\nimport re\nfrom functools import wraps\nfrom typing import Any, AnyStr, Callable, Optional, Pattern\n\nimport streamlit as st\nfrom github import Github, Repository\nfrom google.cloud import firestore\nfrom google.oauth2 import service_account\nfrom networkx import Graph\nfrom pyvis.network import Network\nfrom toolz.functoolz import pipe\n\nfrom python_components.large_lang_model import ai_magic\nfrom python_components.networkx_graphing import get_graphs\nfrom python_components.old_types import Package, T\nfrom src.streamlit_components.graph_visualizer import my_component\n\nst.set_page_config(layout=\'wide\')\nif \'init\' not in st.session_state:\n    st.session_state.init = True\n\n\ndef replace_semicolons_with_new_line(code: str) -> str:\n    return code.replace(\';\', \'\\n\')\n\n\ndef extract_and_remove_pattern(\n    pattern: Pattern[AnyStr], code: str\n) -> tuple[list[str], str]:\n    return pattern.findall(